In [ ]:
!pip install numpy tqdm matplotlib japanize_matplotlib PyQt6

In [14]:
%matplotlib qt

In [15]:
import random
import numpy as np
from tqdm import trange

In [16]:
def simulate_error(n_samples, gps_acc, actual_speed, gps_update_freq=1.3257, return_average=False):
  distance_traveled = actual_speed * gps_update_freq
  speed_error_samples = np.zeros((n_samples, ))
  direction_error_samples = np.zeros((n_samples, ))

  for i in trange(n_samples):
    prev_error_direction = random.random() * 2 * np.pi
    prev_error_length = random.random() * gps_acc

    next_error_direction = random.random() * 2 * np.pi
    next_error_length = random.random() * gps_acc

    prev_point = (prev_error_length * np.cos(prev_error_direction), prev_error_length * np.sin(prev_error_direction))
    next_point = (distance_traveled + next_error_length * np.cos(next_error_direction), next_error_length * np.sin(next_error_direction))

    calculated_speed = np.sqrt((prev_point[0] - next_point[0]) ** 2 + ((prev_point[1] - next_point[1]) ** 2)) / gps_update_freq
    calculated_direction = np.arctan((next_point[1] - prev_point[1])/(next_point[0] - prev_point[0]))

    speed_error_samples[i] = abs(calculated_speed - actual_speed)
    direction_error_samples[i] = abs(calculated_direction)
  if return_average:
    return average(speed_error_samples), average(direction_error_samples)
  return speed_error_samples, direction_error_samples


In [17]:
speed_error, direction_error = simulate_error(1000000, 5.82, 4 / 3.6)

100%|██████████| 1000000/1000000 [00:02<00:00, 391091.41it/s]


In [18]:
def average(arr):
  return sum(arr) / len(arr)

In [19]:
average(speed_error)

np.float64(2.290613864645053)

In [20]:
average(direction_error) / np.pi * 180

np.float64(42.947630438162705)

In [21]:
sample_cruising_speeds = np.arange(0, 3.1, 0.5)

In [22]:
speed_errors = np.zeros_like(sample_cruising_speeds)
direction_errors = np.zeros_like(sample_cruising_speeds)
for i, cruising_speed in enumerate(sample_cruising_speeds):
  speed_error, direction_error = simulate_error(5000000, 5.82, cruising_speed / 3.6, return_average=True)
  speed_errors[i] = speed_error
  direction_errors[i] = direction_error / np.pi * 180

100%|██████████| 5000000/5000000 [00:12<00:00, 411720.35it/s]


In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
plt.plot(sample_cruising_speeds, speed_errors)
plt.xlabel("実走行速度 x(m/s)")
plt.ylabel("計算走行速度誤差 (m/s)")
plt.grid(visible=True)
plt.show()

In [ ]:
plt.plot(sample_cruising_speeds, direction_errors)
plt.xlabel("実走行速度 x(m/s)")
plt.ylabel("計算進行方向誤差 (度)")
plt.grid(visible=True)
plt.show()

In [ ]:
plt.cla()